In [1]:
!pip install tensorflow

In [3]:
!pip install flyr

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from skimage import color, draw
from mpl_toolkits.axes_grid1 import make_axes_locatable
import flyr
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [14]:
import numpy as np
from keras.models import load_model
from PIL import Image

# Load pre-trained YOLOv3 model
model = load_model('/content/drive/MyDrive/full_yolo_backend.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [63]:
def load_dataset_paths(directory):
    dataset_paths = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.json'):
            filepath = os.path.join(directory, filename)
            dataset_paths.append(filepath)
    return dataset_paths

# Function to get mask from annotation data
def get_mask_from_annotation_data(annotation_data, shape=(512, 640)):
    mask = np.zeros(shape, dtype='int16')
    for index, corner_points in enumerate(annotation_data['corners'], start=1):
        polygon = np.array([[coords['y'], coords['x']] for coords in corner_points])
        instance_mask = draw.polygon2mask(shape, polygon)
        mask[instance_mask] = index
    return mask

# Function to visualize image with mask
def visualize_image_with_mask(image_path, annotation_path):
    thermogram = flyr.unpack(image_path)
    annotation_data = get_annotation_data(annotation_path)
    mask = get_mask_from_annotation_data(annotation_data)
    masked_image = color.label2rgb(mask, thermogram.render(palette='grayscale'))

    _, ax = plt.subplots(1, 2, figsize=(10, 5))

    im = ax[0].imshow(thermogram.celsius, cmap='inferno')
    cax = make_axes_locatable(ax[0]).append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax, values=np.unique(thermogram.celsius), label='Temperature (ºC)')
    ax[0].set_title('Thermography image')
    ax[0].set_axis_off()

    ax[1].imshow(masked_image)
    ax[1].set_title('Masked image')
    ax[1].set_axis_off()

    plt.tight_layout()
    plt.show()

# Function to define and train the model
def train_model(images_paths, annotations_paths):
    # Load pre-trained YOLOv3 model
    yolo_model = load_model('/content/drive/MyDrive/full_yolo_backend.h5')

    # Freeze convolutional layers
    for layer in yolo_model.layers:
        layer.trainable = False

    # Extract features from YOLOv3 model
    features = yolo_model.output
    # print("hello")
    # print("features: ",features[1])

    # Add custom classification layers
    x = GlobalAveragePooling2D()(features[1])
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)

    # Create model
    model = Model(inputs=yolo_model.input, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy'])

    # Split data into train and validation sets
    train_images_paths, val_images_paths, train_annotations_paths, val_annotations_paths = \
        train_test_split(images_paths, annotations_paths, test_size=0.2, random_state=42)

    # Define callbacks
    checkpoint_path = 'model_checkpoint.keras'
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, verbose=1)

    # Train the model
    history = model.fit(
        generator(train_images_paths, train_annotations_paths),
        steps_per_epoch=len(train_images_paths),
        epochs=10,
        validation_data=generator(val_images_paths, val_annotations_paths),
        validation_steps=len(val_images_paths),
        callbacks=[checkpoint_callback]
    )

    return model, history

# Function to generate batches of training data
def generator(images_paths, annotations_paths, batch_size=32):
    while True:
        for i in range(0, len(images_paths), batch_size):
            batch_images_paths = images_paths[i:i+batch_size]
            batch_annotations_paths = annotations_paths[i:i+batch_size]
            X_batch = []
            y_batch = []
            for img_path, ann_path in zip(batch_images_paths, batch_annotations_paths):
                thermogram = flyr.unpack(img_path)
                annotation_data = get_annotation_data(ann_path)
                mask = get_mask_from_annotation_data(annotation_data)
                X_batch.append(thermogram.celsius)
                y_batch.append(1 if np.sum(mask) > 0 else 0)  # Label 1 if defect present, else 0
            yield np.array(X_batch)[:, :, :, np.newaxis], np.array(y_batch)

# Main function
def main():
    # Define the path to the dataset
    DATA_PATH = '/content/drive/MyDrive/dataset_1'

    # Load dataset paths
    images_paths = load_dataset_paths("/content/drive/MyDrive/dataset_1/images")
    annotations_paths = load_dataset_paths("/content/drive/MyDrive/dataset_1/annotations")


    # # Visualize an example image with mask
    # example_image_path = images_paths[0]
    # example_annotation_path = annotations_paths[0]
    # visualize_image_with_mask(example_image_path, example_annotation_path)

    # Train the model
    model, history = train_model(images_paths, annotations_paths)

    # Plot training history
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


In [ ]:
if __name__ == "__main__":
    main()